## Twitter Sentiments

In [ ]:
!pip install --upgrade datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.0 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.


In [ ]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.8 MB/s eta 0:00:00


In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
from datasets import load_dataset          #ia-bentebib/tweet_eval_sentiment_fr
twitter_data = load_dataset("mteb/tweet_sentiment_extraction")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/27481 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3534 [00:00<?, ? examples/s]

In [ ]:
twitter_data

DatasetDict({
    train: Dataset({
        features: ['id', 'text', 'label', 'label_text'],
        num_rows: 27481
    })
    test: Dataset({
        features: ['id', 'text', 'label', 'label_text'],
        num_rows: 3534
    })
})

In [ ]:
twitter_df = twitter_data["train"].to_pandas()
twitter_df.head()

,id,text,label,label_text
0,cb774db0d1,"I`d have responded, if I were going",1,neutral
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,0,negative
2,088c60f138,my boss is bullying me...,0,negative
3,9642c003ef,what interview! leave me alone,0,negative
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...",0,negative


In [ ]:
features = twitter_data["train"].features
features

{'id': Value(dtype='string', id=None),
 'text': Value(dtype='string', id=None),
 'label': Value(dtype='int64', id=None),
 'label_text': Value(dtype='string', id=None)}

In [ ]:
twitter_df["label"].value_counts()

,count
label,
1,11118
2,8582
0,7781


In [ ]:
#0 : neg, 1: neutral, 2: positive

In [ ]:
from transformers import AutoTokenizer

model = "microsoft/MiniLM-L12-H384-uncased"
tokenizer = AutoTokenizer.from_pretrained(model)

tokenizer_config.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [ ]:
def tokenize_dataset(data):
  return tokenizer(data["text"], truncation = True)

twitter_tokenized = twitter_data.map(tokenize_dataset, batched = True)

Map:   0%|          | 0/27481 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Map:   0%|          | 0/3534 [00:00<?, ? examples/s]

In [ ]:
twitter_tokenized

DatasetDict({
    train: Dataset({
        features: ['id', 'text', 'label', 'label_text', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 27481
    })
    test: Dataset({
        features: ['id', 'text', 'label', 'label_text', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 3534
    })
})

In [ ]:
twitter_tokenized["train"][7]

{'id': '50e14c0bb8',
 'text': 'Soooo high',
 'label': 1,
 'label_text': 'neutral',
 'input_ids': [101, 17111, 9541, 2152, 102],
 'token_type_ids': [0, 0, 0, 0, 0],
 'attention_mask': [1, 1, 1, 1, 1]}

In [ ]:
import evaluate

accuracy = evaluate.load("accuracy")

In [ ]:
import numpy as np

def get_metric(eval_pred):
  predictions, labels = eval_pred
  predictions = np.argmax(predictions, axis = 1)
  return accuracy.compute(predictions = predictions, references = labels)

In [ ]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

In [ ]:
label2id = {"NEGATIVE": 0, "NEUTRAL": 1,"POSITIVE": 2}
id2label = {0: "NEGATIVE", 1: "NEUTRAL",2: "POSITIVE"}

In [ ]:
model = "microsoft/MiniLM-L12-H384-uncased"
MODEL = AutoModelForSequenceClassification.from_pretrained(model, num_labels = 3, id2label = id2label, label2id = label2id)

pytorch_model.bin:   0%|          | 0.00/133M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at microsoft/MiniLM-L12-H384-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
training_args = TrainingArguments(
    output_dir="tweet_sentiment",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
    eval_strategy="epoch",  # Updated from evaluation_strategy to eval_strategy
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=True,
)


trainer = Trainer(
    model = MODEL,
    args = training_args,
    train_dataset = twitter_tokenized["train"],
    eval_dataset = twitter_tokenized["test"],
    tokenizer = tokenizer,
    compute_metrics = get_metric
)

trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,0.558700,0.513783,0.790889
2,0.484800,0.513678,0.796265


TrainOutput(global_step=3436, training_loss=0.5559063113193712, metrics={'train_runtime': 200.1392, 'train_samples_per_second': 274.619, 'train_steps_per_second': 17.168, 'total_flos': 278499206935404.0, 'train_loss': 0.5559063113193712, 'epoch': 2.0})

In [ ]:
from transformers import AutoTokenizer

model_1 = "microsoft/deberta-base-mnli"
tokenizer_1 = AutoTokenizer.from_pretrained(model_1)

In [ ]:
def tokenize_dataset(data):
  return tokenizer_1(data["text"], truncation = True)

twitter_tokenized_1 = twitter_data.map(tokenize_dataset, batched = True)

Map:   0%|          | 0/27481 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Map:   0%|          | 0/3534 [00:00<?, ? examples/s]

In [ ]:
twitter_tokenized_1

DatasetDict({
    train: Dataset({
        features: ['id', 'text', 'label', 'label_text', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 27481
    })
    test: Dataset({
        features: ['id', 'text', 'label', 'label_text', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 3534
    })
})

In [ ]:
model_1 = "microsoft/deberta-base-mnli"
MODEL_1 = AutoModelForSequenceClassification.from_pretrained(model_1, num_labels = 3, id2label = id2label, label2id = label2id)

pytorch_model.bin:   0%|          | 0.00/557M [00:00<?, ?B/s]

Some weights of the model checkpoint at microsoft/deberta-base-mnli were not used when initializing DebertaForSequenceClassification: ['config']
- This IS expected if you are initializing DebertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
training_args = TrainingArguments(
    output_dir="twitter_sentiment",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
    eval_strategy="epoch",  # Updated from evaluation_strategy to eval_strategy
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=True,
)


trainer = Trainer(
    model = MODEL_1,
    args = training_args,
    train_dataset = twitter_tokenized_1["train"],
    eval_dataset = twitter_tokenized_1["test"],
    tokenizer = tokenizer_1,
    compute_metrics = get_metric
)

trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,0.519900,0.484933,0.805037
2,0.396100,0.509680,0.806452


TrainOutput(global_step=3436, training_loss=0.4745044774864827, metrics={'train_runtime': 704.228, 'train_samples_per_second': 78.046, 'train_steps_per_second': 4.879, 'total_flos': 1261435479121350.0, 'train_loss': 0.4745044774864827, 'epoch': 2.0})

In [ ]:
from transformers import pipeline

In [ ]:
classifier = pipeline("sentiment-analysis", model = "jacobasir/twitter_sentiment")
classifier("Just got promoted! Hard work pays off—feeling on top of the world today!")

config.json:   0%|          | 0.00/950 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/557M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.31k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/969 [00:00<?, ?B/s]

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


[{'label': 'POSITIVE', 'score': 0.6472486853599548}]

In [ ]:
classifier = pipeline("sentiment-analysis", model = "jacobasir/twitter_sentiment")
classifier("Attended a meeting this morning. It was informative and went as expected")

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


[{'label': 'POSITIVE', 'score': 0.992139995098114}]

In [ ]:
classifier = pipeline("sentiment-analysis", model = "jacobasir/twitter_sentiment")
classifier("Missed my flight and lost my luggage. Could this day get any worse?")

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


[{'label': 'NEGATIVE', 'score': 0.9816978573799133}]

In [ ]:
classifier = pipeline("sentiment-analysis", model = "jacobasir/twitter_sentiment")
classifier("I`d have responded, if I were going")

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


[{'label': 'NEUTRAL', 'score': 0.9781225919723511}]

In [ ]:
classifier = pipeline("sentiment-analysis", model = "jacobasir/twitter_sentiment")
classifier("The meeting is scheduled for tomorrow.")

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


[{'label': 'NEUTRAL', 'score': 0.9955247640609741}]